In [1]:
import numpy as np
from numpy import matrix
import sklearn.metrics
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import spearmanr 
import scanpy as sc
import seaborn as sns

In [2]:

#%% Kernel
def kernel(ker, X1, X2, gamma):
    K = None
    if ker == 'linear':
        if X2 is not None:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1), np.asarray(X2))
        else:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1))
    elif ker == 'rbf':
        if X2 is not None:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1), np.asarray(X2), gamma)
        else:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1), None, gamma)
    return K


#%% Kernel Mean Matching (KMM)
class KMM:
    def __init__(self, kernel_type='linear', gamma=1.0, B=1.0, eps=None):
        '''
        Initialization function
        :param kernel_type: 'linear' | 'rbf'
        :param gamma: kernel bandwidth for rbf kernel
        :param B: bound for beta
        :param eps: bound for sigma_beta
        '''
        self.kernel_type = kernel_type
        self.gamma = gamma
        self.B = B
        self.eps = eps

    def fit(self, Xs, Xt):
        '''
        Fit source and target using KMM (compute the coefficients)
        :param Xs: ns * dim
        :param Xt: nt * dim
        :return: Coefficients (Pt / Ps) value vector (Beta in the paper)
        '''
        ns = Xs.shape[0]
        nt = Xt.shape[0]
        if self.eps == None:
            self.eps = self.B / np.sqrt(ns)
        K = kernel(self.kernel_type, Xs, None, self.gamma)
        kappa = np.sum(kernel(self.kernel_type, Xs, Xt, self.gamma) * float(ns) / float(nt), axis=1)

        K = matrix(K)
        kappa = matrix(kappa)
        G = matrix(np.r_[np.ones((1, ns)), -np.ones((1, ns)), np.eye(ns), -np.eye(ns)])
        h = matrix(np.r_[ns * (1 + self.eps), ns * (self.eps - 1), self.B * np.ones((ns,)), np.zeros((ns,))])

        sol = solvers.qp(K, -kappa, G, h)
        beta = np.array(sol['x'])
        return beta

In [3]:
cell=sc.read_h5ad('cell_seurat_100.h5ad')
st=sc.read_h5ad('st_seurat_100.h5ad')

In [4]:
cell

AnnData object with n_obs × n_vars = 1691 × 132
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA'
    var: 'features'

In [5]:
st

AnnData object with n_obs × n_vars = 10000 × 132
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA'
    var: 'features'

In [6]:
cell_X=pd.DataFrame(cell.X.todense())
st_X=pd.DataFrame(st.X.todense())

In [7]:
kmm = KMM(kernel_type='rbf', B=1)
beta = kmm.fit(pd.DataFrame(st.X.todense()), pd.DataFrame(cell.X.todense()))

     pcost       dcost       gap    pres   dres
 0:  5.5621e+06  4.8182e+06  3e+10  1e+01  1e-09
 1:  1.4662e+07  1.7390e+07  4e+09  1e+00  3e-07
 2:  3.4702e+07  2.7866e+07  5e+08  2e-01  2e-08
 3:  4.2801e+07  3.0979e+07  4e+07  1e-02  2e-08
 4:  4.3892e+07  3.9719e+07  5e+06  5e-04  8e-08
 5:  4.3984e+07  4.3820e+07  2e+05  7e-06  1e-07
 6:  4.3981e+07  4.3946e+07  4e+04  1e-06  9e-07
 7:  4.3981e+07  4.3946e+07  4e+04  1e-06  2e-05
 8:  4.3981e+07  4.3946e+07  4e+04  1e-06  2e-05
 9:  4.3981e+07  4.3946e+07  4e+04  1e-06  3e-05
10:  4.3981e+07  4.3946e+07  4e+04  1e-06  4e-05
11:  4.3980e+07  4.3946e+07  3e+04  1e-06  4e-05
12:  4.3980e+07  4.3946e+07  3e+04  1e-06  5e-05
13:  4.3979e+07  4.3947e+07  3e+04  8e-07  4e-05
14:  4.3978e+07  4.3947e+07  3e+04  6e-07  3e-05
15:  4.3977e+07  4.3948e+07  3e+04  5e-07  4e-05
16:  4.3976e+07  4.3950e+07  3e+04  4e-07  2e-05
17:  4.3975e+07  4.3952e+07  2e+04  3e-07  3e-05
18:  4.3974e+07  4.3954e+07  2e+04  2e-07  9e-06
19:  4.3973e+07  4.39

In [8]:
beta=pd.DataFrame(beta)
beta.to_csv('beta_100.csv',index= False, header=0)

In [9]:
beta

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
9995,1.0
9996,1.0
9997,1.0
9998,1.0
